In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
// Create network
val model = Network("model")

// Block 1: nodes
val node = arrayOfNulls<Node>(3)
node[0] = Delay(model, "Delay")
node[1] = Queue(model, "Queue1", SchedStrategy.PS)
node[2] = Queue(model, "Queue2", SchedStrategy.PS)

// Block 2: classes (1 job each)
val jobclass = arrayOfNulls<ClosedClass>(2)
jobclass[0] = ClosedClass(model, "Class1", 1, node[0]!!, 0)
jobclass[1] = ClosedClass(model, "Class2", 1, node[0]!!, 0)

println("Network created with 3 nodes and 2 classes (1 job each)")
println("  ${node[0]!!.name}: Delay node")
println("  ${node[1]!!.name}: PS Queue")
println("  ${node[2]!!.name}: PS Queue")

In [ ]:
// Set advanced service distributions
println("=== Setting Advanced Service Distributions ===")

// Delay node - mixed distributions
node[0]!!.setService(jobclass[0]!!, Erlang(3.0, 2))  // Erlang with rate=3, shape=2
node[0]!!.setService(jobclass[1]!!, HyperExp(0.5, 3.0, 10.0))     // HyperExp with prob=0.5, rates=3.0,10.0

println("Delay node distributions:")
println("  Class1: Erlang(rate=3, shape=2) - Low variability")
println("  Class2: HyperExp(p=0.5, λ1=3, λ2=10) - High variability")

// Queue1 - HyperExp and MMPP2
node[1]!!.setService(jobclass[0]!!, HyperExp(0.1, 1.0, 10.0))     // HyperExp with prob=0.1, rates=1.0,10.0
node[1]!!.setService(jobclass[1]!!, MMPP2(1.0, 2.0, 3.0, 4.0))           // MMPP2 with parameters λ0=1, λ1=2, σ01=3, σ10=4

println("\nQueue1 distributions:")
println("  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Very high variability")
println("  Class2: MMPP2(λ0=1, λ1=2, σ01=3, σ10=4) - Markov Modulated Poisson Process")

// Queue2 - HyperExp and Erlang
node[2]!!.setService(jobclass[0]!!, HyperExp(0.1, 1.0, 10.0))     // Same as Queue1 for Class1
node[2]!!.setService(jobclass[1]!!, Erlang(1.0, 2))                 // Erlang with rate=1, shape=2

println("\nQueue2 distributions:")
println("  Class1: HyperExp(p=0.1, λ1=1, λ2=10) - Same as Queue1")
println("  Class2: Erlang(rate=1, shape=2) - Low variability")

In [ ]:
// Set up connectivity
model.addLink(node[0], node[0])  // Delay can route to itself
model.addLink(node[0], node[1])  // Delay -> Queue1
model.addLink(node[0], node[2])  // Delay -> Queue2
model.addLink(node[1], node[0])  // Queue1 -> Delay
model.addLink(node[2], node[0])  // Queue2 -> Delay

println("\n=== Configuring Routing Strategies ===")

// Class1: Probabilistic routing from Delay
node[0]!!.setProbRouting(jobclass[0]!!, node[0]!!, 0.0)  // No self-loop for Class1
node[0]!!.setProbRouting(jobclass[0]!!, node[1]!!, 0.3)  // 30% to Queue1
node[0]!!.setProbRouting(jobclass[0]!!, node[2]!!, 0.7)  // 70% to Queue2
node[1]!!.setProbRouting(jobclass[0]!!, node[0]!!, 1.0)  // Queue1 -> Delay (deterministic)
node[2]!!.setProbRouting(jobclass[0]!!, node[0]!!, 1.0)  // Queue2 -> Delay (deterministic)

println("Class1 routing (Probabilistic):")
println("  From Delay: 30% Queue1, 70% Queue2")
println("  From Queues: 100% back to Delay")

// Class2: Random routing strategy
node[0]!!.setRouting(jobclass[1]!!, RoutingStrategy.RAND)  // Random from Delay
node[1]!!.setRouting(jobclass[1]!!, RoutingStrategy.RAND)  // Random from Queue1
node[2]!!.setRouting(jobclass[1]!!, RoutingStrategy.RAND)  // Random from Queue2

println("\nClass2 routing (Random):")
println("  All nodes use random routing among connected destinations")

In [ ]:
// Link the routing matrix
val P = model.initRoutingMatrix()
model.link(P)

println("Routing matrix linked successfully")
println("\nModel configuration complete!")

In [ ]:
// Configure solver (using JMT as it handles advanced distributions well)
println("=== Solving with JMT (best for advanced distributions) ===")

val solver = JMT(model, "seed", 23000, "verbose", 1, "samples", 20000)  // Higher samples for better accuracy with complex distributions

println("Solver configured with 20000 samples for higher accuracy")

In [ ]:
// Solve the model
println("\n=== SOLVER: ${solver.name.replace("Solver", "")} ===")
val avgTable = solver.avgTable
avgTable.print()

In [ ]:
// Summary insights
println("\n=== Key Insights ===")

val totalTput = avgTable.getTput().elementSum()
val totalQlen = avgTable.getQLen().elementSum()

println("\n**System Overview:**")
println("- Total system throughput: ${String.format("%.4f", totalTput)}")
println("- Total queue length: ${String.format("%.4f", totalQlen)}")
println("- Both classes have 1 job each (closed system)")

println("\n**Distribution Modeling Benefits:**")
println("- Erlang: Models services with low variability (e.g., automated processes)")
println("- HyperExp: Models services with high variability (e.g., complex tasks)")
println("- MMPP2: Models bursty arrival/service patterns (e.g., network traffic)")

println("\n**Routing Strategy Impact:**")
println("- Class1: Deterministic splits provide predictable load distribution")
println("- Class2: Random routing provides automatic load balancing")

println("\n**Solver Recommendation:**")
println("- JMT handles all distribution types through simulation")
println("- Some analytical solvers may have limitations with advanced distributions")
println("- Higher sample counts improve accuracy for complex distributions")